<a href="https://colab.research.google.com/github/akybharat/TSAI-Assignments/blob/main/TorchText_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from torchtext.datasets import IMDB

In [54]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer


In [55]:
from torch.utils.data import DataLoader

In [56]:
import torch

In [57]:
from torch import nn

In [58]:
train_iter = IMDB(split='train')

In [59]:
tokenizer = get_tokenizer("basic_english")

In [60]:
def get_token(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

In [61]:
text_pipeline = lambda x: vocab_out(tokenizer(x))
label_pipeline = lambda x: 0 if x=='neg' else 1

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [63]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [64]:
vocab_out = build_vocab_from_iterator(get_token(train_iter), specials=["<unk>"],  min_freq= 5)

In [65]:
vocab_out.set_default_index(vocab_out['<unk>'])

In [90]:
class TextClassification(nn.Module):
      def __init__(self, vocab_size, embedding_dim, num_classes):
          super(TextClassification, self).__init__()
          self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim)

          # self.encoder = nn.LSTM(64, 
          #                  32,
          #                  num_layers=2,
          #                  batch_first=True)
          
          #self.fc = nn.Linear(embedding_dim, num_classes)
          self.fc1 = nn.Linear(embedding_dim, 16)
          self.fc2 = nn.Linear(16, num_classes)
          self.init_weight()
      def init_weight(self):
          initrange = 0.5
          self.embedding.weight.data.uniform_(-initrange, initrange)
          # self.fc.weight.data.uniform_(-initrange, initrange)
          # self.fc.bias.data.zero_()
          self.fc1.weight.data.uniform_(-initrange, initrange)
          self.fc1.bias.data.zero_()
          self.fc2.weight.data.uniform_(-initrange, initrange)
          self.fc2.bias.data.zero_()
      
      def forward(self, text, offset):
          out = self.embedding(text, offset)
          #out = self.encoder(out)
          out = torch.relu(self.fc1(out))
          return self.fc2(out)

In [91]:
# train_iter = IMDB(split='train')
# vocab_size = len(vocab_out)
# embedding_dim = 64
# num_classes = len(set([label for label, _ in train_iter]))

model = TextClassification(vocab_size, embedding_dim, num_classes)

In [70]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [94]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

In [95]:
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  8.18s | valid accuracy    0.782 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  8.05s | valid accuracy    0.827 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  8.24s | valid accuracy    0.822 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  8.32s | valid accuracy    0.878 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  8.32s | valid accuracy    0.878 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  8.35s |